# Library

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# Data Loading

In [ ]:
train = pd.read_csv('../data/train.csv', index_col=0)
test = pd.read_csv('../data/test.csv', index_col=0)
sample_submit = pd.read_csv('../data/sample_submission.csv', index_col=0, header=None)

# Check

In [2]:
train.dtypes.index

NameError: name 'train' is not defined

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

# preprocessing

In [ ]:
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']].head()

In [ ]:
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']]= \
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', ''))

test[['DisbursementGross',  'GrAppv', 'SBA_Appv']]= \
test[['DisbursementGross',  'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', ''))
  
train[['DisbursementGross',  'GrAppv', 'SBA_Appv']].head()

In [ ]:
train.fillna('NULL', inplace=True)

# EDA

- numeric

In [ ]:
numeric_columns = train.select_dtypes(include=['number']).columns

- corr

In [ ]:
corr_matrix = train[numeric_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Standardized Columns')
plt.show()

# Model

- 量的変数のみ使用

In [ ]:
train_numeric = train[numeric_columns]

- データ分割

In [ ]:
X = train_numeric.drop('MIS_Status', axis=1)
y = train_numeric['MIS_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

- LightGBMパラメータ無し

In [ ]:
lgbm = lgb.LGBMClassifier()

- model学習

In [ ]:
lgbm.fit(X_train, y_train)

- val確認

In [ ]:
y_pred = lgbm.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

- F1スコア確認

In [ ]:
mean_f1 = f1_score(y_test, y_pred, average='macro')
print(f'Mean F1 Score: {mean_f1}')

- pred作成

In [ ]:
predictions = lgbm.predict(test[numeric_columns.drop('MIS_Status')] )

- submit作成

In [ ]:
sample_submit[1] = predictions
sample_submit.to_csv('submit_tutorial.csv', header=None)